In [5]:
import os
import json
import sqlite3
import voyageai
import anthropic


import numpy as np
import pandas as pd

from dotenv import load_dotenv
from chromadb.utils import embedding_functions
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [7]:
class AnthropicCalls:
    def __init__(
            self,
            name="Anthropic Chat",
            api_key="",
            model="claude-3-5-sonnet-20240620",
            max_tokens=1024,
            temperature=0.7,
            system_prompt="",
            stream=False,
    ):
        self.name = name
        self.api_key = api_key
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.system_prompt = system_prompt
        self.stream = stream
        self.history = []

        self.client = anthropic.Anthropic(
            api_key=self.api_key,
        )

    def add_message(self, role, content):
        self.history.append(
            {
                "role": role, 
                "content": content
            }
        )
    
    def clear_history(self):
        self.history.clear()

    def chat(self, message, **kwargs):
        self.add_message("user", message)
        return self.get_response(**kwargs)
        
    def get_response(self, should_print=True, **kwargs) -> str:
        params = {
            "model": self.model,
            "max_tokens": self.max_tokens,
            "temperature": self.temperature,
            "messages": self.history,
            "system": self.system_prompt,
            **kwargs
        }
        assistant_response = ""
        text_response = ""

        if self.stream:
            with self.client.messages.stream(
                **params
            ) as stream:
                for text_chunk in stream.text_stream:
                    text_response += str(text_chunk)
                    if should_print:
                        print(text_chunk, end="", flush=True)
                assistant_response = stream.get_final_message()
        else:
            assistant_response = self.client.messages.create(
                **params
            )
            text_response = assistant_response.content[0].text
            if should_print:
                print(text_response, end="")

        if should_print:
            print()

        self.add_message("assistant", text_response)
        return assistant_response


In [4]:
calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY)

calls.chat(
    message="Hi!",
)


Hello! How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything.


Message(id='msg_01BFDZnACoLyCsGXSBdmNofm', content=[TextBlock(text='Hello! How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything.', type='text')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=9, output_tokens=30))

In [ ]:
calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY, stream=True)

calls.chat(
    message="Write a Long abstract",
)


In [ ]:
calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY, stream=True)

n = 20
message = ""
while message != "END": 
    message = input("User:")
    calls.chat(message=f"{message} please use {n} words or less")


In [ ]:
calls.history

Let's implement RAG!

In [8]:
class SQLiteCalls:
    def __init__(
            self,
            db_path="sqlite.db"
    ):
        self.db_path = db_path
        self.setup_database()

    def setup_database(self):
        conn = sqlite3.connect(self.db_path)

        # cursor = conn.cursor()
        # cursor.execute('DROP TABLE IF EXISTS chat_history')
        
        conn.execute("""
            CREATE TABLE IF NOT EXISTS chat_history (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                role TEXT,
                message TEXT,
                embedding TEXT,
                date TEXT DEFAULT CURRENT_TIMESTAMP
            );
        """)
        conn.commit()
        conn.close()

    def save_message(self, role: str, message: str, embedding):
        conn = sqlite3.connect(self.db_path, timeout=5)
        try:
            cursor = conn.cursor()
            embedding_str = json.dumps(embedding) if not isinstance(embedding, str) else embedding

            cursor.execute("""
                INSERT INTO chat_history (role, message, embedding)
                VALUES (?, ?, ?)
            """, (role, message, embedding_str))

            conn.commit()
        finally:
            conn.close()

    def load_chat_to_dataframe(self, role=""):
        conn = sqlite3.connect(self.db_path, timeout=5)
        try:
            if role == "user" or role == "assistant":
                df = pd.read_sql_query('''
                    SELECT role, message, embedding, date FROM chat_history
                    WHERE role = ? 
                    ORDER BY date ASC                
                ''', conn, params=(role, ))
            else:
                df = pd.read_sql_query('''
                    SELECT * FROM chat_history
                    ORDER BY date ASC                
                ''', conn)
        finally:
            conn.close()
        return df


Adding Embedder

In [9]:
class AnthropicCalls:
    def __init__(
            self,
            name="Anthropic Chat",
            api_key="",
            model="claude-3-5-sonnet-20240620",
            max_tokens=1024,
            temperature=0.7,
            system_prompt="",
            stream=False,
    ):
        self.name = name
        self.api_key = api_key
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.system_prompt = system_prompt
        self.stream = stream
        self.history = []

        self.client = anthropic.Anthropic(
            api_key=self.api_key,
        )

        # self.embeder = voyageai.Client(
        #     api_key=self.api_key,
        # )

        self.embeder = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

    def add_message(self, role, content):
        self.history.append(
            {
                "role": role, 
                "content": content
            }
        )
    
    def clear_history(self):
        self.history.clear()

    def chat(self, message, clear_after_response=False, **kwargs) -> str:
        self.add_message("user", message)
        response = self.get_response(**kwargs)
        
        if clear_after_response:
            self.clear_history()
        return response
        
    def get_response(self, should_print=True, **kwargs) -> str:
        params = {
            "model": self.model,
            "max_tokens": self.max_tokens,
            "temperature": self.temperature,
            "messages": self.history,
            "system": self.system_prompt,
            **kwargs
        }
        assistant_response = ""
        text_response = ""

        if self.stream:
            with self.client.messages.stream(
                **params
            ) as stream:
                for text_chunk in stream.text_stream:
                    text_response += str(text_chunk)
                    if should_print:
                        print(text_chunk, end="", flush=True)
                assistant_response = stream.get_final_message()
        else:
            assistant_response = self.client.messages.create(
                **params
            )
            text_response = assistant_response.content[0].text
            if should_print:
                print(text_response, end="")

        if should_print:
            print()

        self.add_message("assistant", text_response)
        return assistant_response
        
    def get_embedding(self, text):
        text = text.replace("\n", " ")
        # return self.embeder.embed(
        #     texts=[text],
        #     model=model
        # ).embeddings[0]
        return self.embeder([text])[0]


In [ ]:
LLM_calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY)

LLM_calls.get_embedding("When is Apple's conference call scheduled?")

In [10]:
LLM_calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY, stream=True)
SQL_calls = SQLiteCalls()


def get_context(embedding, role="", n=1):
    chat_df = SQL_calls.load_chat_to_dataframe(role)
    context = find_top_n_similar(chat_df, embedding, n)
    return context


def find_top_n_similar(df, user_input_embedding, n=5):
    if df.empty or 'embedding' not in df.columns:
        print("The DataFrame is empty or missing the 'embedding' column.")
        return pd.DataFrame()
    
    df['embedding'] = df['embedding'].apply(
        lambda emb: json.loads(emb) if isinstance(emb, str) else emb
    )
    df['similarity'] = df['embedding'].apply(
        lambda emb: similarty_search(user_input_embedding, emb)
    )

    top_n_df = df.sort_values(by='similarity', ascending=False).head(n)
    # To have messages in the correct order
    top_n_df = top_n_df.sort_values(by='date', ascending=True)

    return top_n_df


def similarty_search(embedding1, embedding2):
    embedding1 = np.array(embedding1).reshape(1, -1)
    embedding2 = np.array(embedding2).reshape(1, -1)

    similarity = cosine_similarity(embedding1, embedding2)

    return similarity[0][0]


def send_message(text: str, clear_after_response=False) -> str:
    embedding = LLM_calls.get_embedding(text)
    context = get_context(embedding, role="user", n=3)
    SQL_calls.save_message(
        role="user",
        message=text,
        embedding=embedding
    )

    if context.empty:
        print("Context is empty")
        combined_message = text
    else:
        context_messages = context["message"].tolist()
        print("------", "Context:", *context_messages, "------", sep="\n")
        context_message = '\n'.join(context_messages)
        combined_message = f"Provided context:\n{context_message}\nUser message:\n{text}"    

    llm_response = LLM_calls.chat(combined_message, clear_after_response).content[0].text
    llm_embedding = LLM_calls.get_embedding(llm_response)
    SQL_calls.save_message(
        role="assistant",
        message=llm_response, 
        embedding=llm_embedding
    )

    return llm_response

/Users/kanstantsinnikalayenka/Documents/Obsidian Vault/projects/Study/LH_RAG_WORKSHOP_2024/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/kanstantsinnikalayenka/Documents/Obsidian Vault/projects/Study/LH_RAG_WORKSHOP_2024/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
def conversation():
    message = ''

    while message != "END":
        message = input("User: ")
        print("------", message, "------", sep='\n')
        if message != "END":
            response = send_message(message, clear_after_response=True)

        # LLM_calls.clear_history() # clear history of conversation

In [ ]:
conversation()

In [ ]:
LLM_calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY)

get_context(
    embedding = LLM_calls.get_embedding("What is my name?"), 
    role="user", 
    n=3
) 
# ['message'] .to_list()

In [ ]:
conversation()

So it is not really convenient
(mess with questions, saving everything)

In [12]:
SQL_calls = SQLiteCalls("key_words.db")


def send_message(text: str, clear_after_response=False) -> str:
    key_words = ['remember', 'memorize', 'learn']

    embedding = LLM_calls.get_embedding(text)
    context = get_context(embedding, role="user", n=2)

    if any([ word in text.lower() for word in key_words ]): # Now we are saving only messages with key words
        SQL_calls.save_message(
            role="user",
            message=text,
            embedding=embedding
        )

    if context.empty:
        print("Context is empty")
        combined_message = text
    else:
        context_messages = context["message"].tolist()
        print("Context:", *context_messages, "------", sep="\n")
        context_message = '\n'.join(context_messages)
        combined_message = f"Provided context:\n{context_message}\nUser message:\n{text}"    

    llm_response = LLM_calls.chat(combined_message, clear_after_response)
    
    return llm_response

In [ ]:
conversation()

Now it's somewhat valid, but still have some issues (provides useless context when there is nothing else to give)

Let's add LLM determination if context is relevant

In [13]:
context_determinator = AnthropicCalls(
    api_key=ANTHROPIC_API_KEY, 
    max_tokens=400,
    system_prompt="You are a helpful assistant that determines if a chunk of text is relevant to a given query.\n" +
        "Respond with JSON object containing a boolean 'is_relevant' field and a 'reason' field explaining your decision"
)


def is_relevant(chunk: str, query: str):
    response = context_determinator.chat(
        f"Query: {query}\n\nChunk: {chunk}\n\nIs this chunk relevant to the query? Respond in JSON format.",
        should_print=False,
        clear_after_response=True
    )
    print("Chunk:\n", chunk)
    print("Response:\n", response.content[0].text)
    return json.loads(response.content[0].text)["is_relevant"]


def send_message(text: str, clear_after_response=False) -> str:
    key_words = ['remember', 'memorize', 'learn']

    embedding = LLM_calls.get_embedding(text)
    context = get_context(embedding, role="user", n=3)

    if any([ word in text.lower() for word in key_words ]):
        SQL_calls.save_message(
            role="user",
            message=text,
            embedding=embedding
        )

    if context.empty:
        print("Context is empty")
        combined_message = text
    else: # Now we will check if context we get is relevant to our message
        context_messages = context["message"].tolist()
        cleared_context = []
        for context_chunk in context_messages:
            if is_relevant(context_chunk, text):
                cleared_context.append(context_chunk)

        # If there are any chunks left:
        if len(cleared_context) > 0:
            print("Context:", *cleared_context, "------", sep="\n")
            context_message = '\n'.join(cleared_context)
            combined_message = f"Provided context:\n{context_message}\nUser message:\n{text}"    
        else:
            combined_message = text

    llm_response = LLM_calls.chat(combined_message, clear_after_response)
    
    return llm_response

In [ ]:
conversation()

Cool! That's already much better!

Now let's head to other option

User data extractor

In [14]:
user_data_extractor = AnthropicCalls(
    api_key=ANTHROPIC_API_KEY, 
    max_tokens=1024,
    system_prompt="You are a helpful assistant that extracts chunk of user related data from given query.\n"
)

extractor_tool = {
    "name": "save_extracted_data",
    "description": "Save a given data extracted from the users query.",
    "input_schema": {
        "type": "object",
        "properties": {
            "chunk": {
                "type": "string",
                "description": "Extracted data."
            }
        },
        "required": ["chunk"]
    }
}


def extract_user_data(query: str):
    response = user_data_extractor.chat(
        f"Query: {query}\n\nExtracts user related data from this query? Use tools to save it.",
        should_print=False,
        tools = [extractor_tool],
        clear_after_response=True
    )
    if response.stop_reason == "tool_use":
        for item in response.content:
            if item.type == "tool_use":
                if item.name == "save_extracted_data":
                    print("\nTool_use: ", item)
                    print("------")
                    return item.input.get("chunk", False)
    # print("Response:\n", response)
    return False


def send_message(text: str, clear_after_response=False) -> str:
    # key_words = ['remember', 'memorize', 'learn']

    embedding = LLM_calls.get_embedding(text)
    context = get_context(embedding, role="user", n=3)

    extracted_data = extract_user_data(text)
    if extracted_data:
        SQL_calls.save_message(
            role="user",
            message=str(extracted_data),
            embedding=embedding
        )

    if context.empty:
        print("Context is empty")
        combined_message = text
    else: # Now we will check if context we get is relevant to our message
        context_messages = context["message"].tolist()
        cleared_context = []
        for context_chunk in context_messages:
            if is_relevant(context_chunk, text):
                cleared_context.append(context_chunk)

        # If there are any chunks left:
        if len(cleared_context) > 0:
            print("Context:", *cleared_context, "------", sep="\n")
            context_message = '\n'.join(cleared_context)
            combined_message = f"Provided context:\n{context_message}\nUser message:\n{text}"    
        else:
            combined_message = text

    llm_response = LLM_calls.chat(combined_message, clear_after_response)
    
    return llm_response

In [ ]:
conversation()

One more trick and exercise

In [15]:
article = """
Atom
Helium atom ground state
An illustration of the helium atom, depicting the nucleus (pink) and the electron cloud distribution (black). The nucleus (upper right) in helium-4 is in reality spherically symmetric and closely resembles the electron cloud, although for more complicated nuclei this is not always the case. The black bar is one angstrom (10−10 m or 100 pm).
Classification
Smallest recognized division of a chemical element
Properties
Mass range	1.67×10−27 to 4.52×10−25 kg
Electric charge	zero (neutral), or ion charge
Diameter range	62 pm (He) to 520 pm (Cs) (data page)
Components	Electrons and a compact nucleus of protons and neutrons
Atoms are the basic particles of the chemical elements. An atom consists of a nucleus of protons and generally neutrons, surrounded by an electromagnetically bound swarm of electrons. The chemical elements are distinguished from each other by the number of protons that are in their atoms. For example, any atom that contains 11 protons is sodium, and any atom that contains 29 protons is copper. Atoms with the same number of protons but a different number of neutrons are called isotopes of the same element.

Atoms are extremely small, typically around 100 picometers across. A human hair is about a million carbon atoms wide. Atoms are smaller than the shortest wavelength of visible light, which means humans cannot see atoms with conventional microscopes. They are so small that accurately predicting their behavior using classical physics is not possible due to quantum effects.

More than 99.94% of an atom's mass is in the nucleus. Protons have a positive electric charge and neutrons have no charge, so the nucleus is positively charged. The electrons are negatively charged, and this opposing charge is what binds them to the nucleus. If the numbers of protons and electrons are equal, as they normally are, then the atom is electrically neutral as a whole. If an atom has more electrons than protons, then it has an overall negative charge, and is called a negative ion (or anion). Conversely, if it has more protons than electrons, it has a positive charge, and is called a positive ion (or cation).

The electrons of an atom are attracted to the protons in an atomic nucleus by the electromagnetic force. The protons and neutrons in the nucleus are attracted to each other by the nuclear force. This force is usually stronger than the electromagnetic force that repels the positively charged protons from one another. Under certain circumstances, the repelling electromagnetic force becomes stronger than the nuclear force. In this case, the nucleus splits and leaves behind different elements. This is a form of nuclear decay.

Atoms can attach to one or more other atoms by chemical bonds to form chemical compounds such as molecules or crystals. The ability of atoms to attach and detach from each other is responsible for most of the physical changes observed in nature. Chemistry is the science that studies these changes.

History of atomic theory
Main article: History of atomic theory
In philosophy
Main article: Atomism
The basic idea that matter is made up of tiny indivisible particles is an old idea that appeared in many ancient cultures. The word atom is derived from the ancient Greek word atomos,[a] which means "uncuttable". But this ancient idea was based in philosophical reasoning rather than scientific reasoning. Modern atomic theory is not based on these old concepts.[1][2] In the early 19th century, the scientist John Dalton (1766–1844) found evidence that matter really is composed of discrete units, and so applied the word atom to those units.[3]

Dalton's law of multiple proportions

Various atoms and molecules from A New System of Chemical Philosophy (John Dalton 1808).
In the early 1800s, John Dalton compiled experimental data gathered by him and other scientists and discovered a pattern now known as the "law of multiple proportions". He noticed that in any group of chemical compounds which all contain two particular chemical elements, the amount of Element A per measure of Element B will differ across these compounds by ratios of small whole numbers. This pattern suggested that each element combines with other elements in multiples of a basic unit of weight, with each element having a unit of unique weight. Dalton decided to call these units "atoms".[4]

For example, there are two types of tin oxide: one is a grey powder that is 88.1% tin and 11.9% oxygen, and the other is a white powder that is 78.7% tin and 21.3% oxygen. Adjusting these figures, in the grey powder there is about 13.5 g of oxygen for every 100 g of tin, and in the white powder there is about 27 g of oxygen for every 100 g of tin. 13.5 and 27 form a ratio of 1:2. Dalton concluded that in the grey oxide there is one atom of oxygen for every atom of tin, and in the white oxide there are two atoms of oxygen for every atom of tin (SnO and SnO2).[5][6]

Dalton also analyzed iron oxides. There is one type of iron oxide that is a black powder which is 78.1% iron and 21.9% oxygen; and there is another iron oxide that is a red powder which is 70.4% iron and 29.6% oxygen. Adjusting these figures, in the black powder there is about 28 g of oxygen for every 100 g of iron, and in the red powder there is about 42 g of oxygen for every 100 g of iron. 28 and 42 form a ratio of 2:3. Dalton concluded that in these oxides, for every two atoms of iron, there are two or three atoms of oxygen respectively (Fe2O2 and Fe2O3).[b][7][8]

As a final example: nitrous oxide is 63.3% nitrogen and 36.7% oxygen, nitric oxide is 44.05% nitrogen and 55.95% oxygen, and nitrogen dioxide is 29.5% nitrogen and 70.5% oxygen. Adjusting these figures, in nitrous oxide there is 80 g of oxygen for every 140 g of nitrogen, in nitric oxide there is about 160 g of oxygen for every 140 g of nitrogen, and in nitrogen dioxide there is 320 g of oxygen for every 140 g of nitrogen. 80, 160, and 320 form a ratio of 1:2:4. The respective formulas for these oxides are N2O, NO, and NO2.[9][10]

Discovery of the electron
In 1897, J. J. Thomson discovered that cathode rays are not a form of light but made of negatively charged particles because they can be deflected by electric and magnetic fields.[11] He measured these particles to be at least a thousand times lighter than hydrogen (the lightest atom).[12] He called these new particles corpuscles but they were later renamed electrons since these are the particles that carry electricity.[13] Thomson also showed that electrons were identical to particles given off by photoelectric and radioactive materials.[14] Thomson explained that an electric current is the passing of electrons from one atom to the next, and when there was no current the electrons embedded themselves in the atoms. This in turn meant that atoms were not indivisible as scientists thought. The atom was composed of electrons whose negative charge was balanced out by some source of positive charge to create an electrically neutral atom. Ions, Thomson explained, must be atoms which have an excess or shortage of electrons.[15]

Discovery of the nucleus

The Rutherford scattering experiments:
Left: All the alpha particles should have passed the atom with negligible deflection.
Right: A small portion of the particles were heavily deflected by the concentrated charge of the nucleus.
Main article: Rutherford scattering experiments
The electrons in the atom logically had to be balanced out by a commensurate amount of positive charge, but Thomson had no idea where this positive charge came from, so he tentatively proposed that this positive charge was everywhere in the atom, the atom being in the shape of a sphere. Following from this, he imagined the balance of electrostatic forces would distribute the electrons throughout the sphere in a more or less even manner.[16] Thomson's model is popularly known as the plum pudding model, though neither Thomson nor his colleagues used this analogy.[17] Thomson's model was incomplete, it was unable to predict any other properties of the elements such as emission spectra and valencies. It was soon rendered obsolete by the discovery of the atomic nucleus.

Between 1908 and 1913, Ernest Rutherford and his colleagues Hans Geiger and Ernest Marsden performed a series of experiments in which they bombarded thin foils of metal with a beam of alpha particles. They did this to measure the scattering patterns of the alpha particles. They spotted a small number of alpha particles being deflected by angles greater than 90°. This shouldn't have been possible according to the Thomson model of the atom, whose charges were too diffuse to produce a sufficiently strong electric field. The deflections should have all been negligible. Rutherford proposed that the positive charge of the atom along with most of the atom's mass is concentrated in a tiny nucleus at the center of the atom. Only such an intense concentration of positive charge, anchored by its high mass and separated from the negative charge, could produce an electric field that could deflect the alpha particles so strongly.[18]

Bohr model
Main article: Bohr model

The Bohr model of the atom, with an electron making instantaneous "quantum leaps" from one orbit to another with gain or loss of energy. This model of electrons in orbits is obsolete.
A problem in classical mechanics is that an accelerating charged particle radiates electromagnetic radiation, causing the particle to lose kinetic energy. Circular motion counts as acceleration, which means that an electron orbiting a central charge should spiral down into the nucleus as it loses speed. In 1913, the physicist Niels Bohr proposed a new model in which the electrons of an atom were assumed to orbit the nucleus but could only do so in a finite set of orbits, and could jump between these orbits only in discrete changes of energy corresponding to absorption or radiation of a photon.[19] This quantization was used to explain why the electrons' orbits are stable and why elements absorb and emit electromagnetic radiation in discrete spectra.[20] Bohr's model could only predict the emission spectra of hydrogen, not atoms with more than one electron.

Discovery of protons and neutrons
Main articles: Atomic nucleus and Discovery of the neutron
Back in 1815, William Prout observed that the atomic weights of many elements were multiples of hydrogen's atomic weight, which is in fact true for all of them if one takes isotopes into account. In 1898, J. J. Thomson found that the positive charge of a hydrogen ion is equal to the negative charge of an electron.[21] In 1913, Henry Moseley discovered that the frequencies of X-ray emissions from an excited atom were a mathematical function of its atomic number and hydrogen's nuclear charge. In 1917 Rutherford bombarded nitrogen gas with alpha particles and detected hydrogen ions being emitted from the gas, and concluded that they were produced by alpha particles hitting and splitting the nitrogen atoms.[22]

These observations led Rutherford to conclude that the hydrogen nucleus is a singular particle with a positive charge equal to the electron's negative charge.[23] He named this particle "proton" in 1920.[24] An element's atomic number, which is defined as the element's position on the periodic table, is also the number of protons it has in its nucleus. The atomic weight of each element is higher than its proton number, so Rutherford hypothesized that the surplus weight was carried by unknown particles with no electric charge and a mass equal to that of the proton.

In 1928, Walter Bothe observed that beryllium emitted a highly penetrating, electrically neutral radiation when bombarded with alpha particles. It was later discovered that this radiation could knock hydrogen atoms out of paraffin wax. Initially it was thought to be high-energy gamma radiation, since gamma radiation had a similar effect on electrons in metals, but James Chadwick found that the ionization effect was too strong for it to be due to electromagnetic radiation, so long as energy and momentum were conserved in the interaction. In 1932, Chadwick exposed various elements, such as hydrogen and nitrogen, to the mysterious "beryllium radiation", and by measuring the energies of the recoiling charged particles, he deduced that the radiation was actually composed of electrically neutral particles which could not be massless like the gamma ray, but instead were required to have a mass similar to that of a proton. Chadwick now claimed these particles as Rutherford's neutrons.[25]

The current consensus model

The modern model of atomic orbitals draws zones where an electron is most likely to be found at any moment.
In 1925, Werner Heisenberg published the first consistent mathematical formulation of quantum mechanics (matrix mechanics).[26] One year earlier, Louis de Broglie had proposed that all particles behave like waves to some extent,[27] and in 1926 Erwin Schroedinger used this idea to develop the Schroedinger equation, which describes electrons as three-dimensional waveforms rather than points in space.[28] A consequence of using waveforms to describe particles is that it is mathematically impossible to obtain precise values for both the position and momentum of a particle at a given point in time. This became known as the uncertainty principle, formulated by Werner Heisenberg in 1927.[26] In this concept, for a given accuracy in measuring a position one could only obtain a range of probable values for momentum, and vice versa.[29] Thus, the planetary model of the atom was discarded in favor of one that described atomic orbital zones around the nucleus where a given electron is most likely to be found.[30][31] This model was able to explain observations of atomic behavior that previous models could not, such as certain structural and spectral patterns of atoms larger than hydrogen.

Structure
Subatomic particles
Main article: Subatomic particle
Though the word atom originally denoted a particle that cannot be cut into smaller particles, in modern scientific usage the atom is composed of various subatomic particles. The constituent particles of an atom are the electron, the proton and the neutron.

The electron is the least massive of these particles by four orders of magnitude at 9.11×10−31 kg, with a negative electrical charge and a size that is too small to be measured using available techniques.[32] It was the lightest particle with a positive rest mass measured, until the discovery of neutrino mass. Under ordinary conditions, electrons are bound to the positively charged nucleus by the attraction created from opposite electric charges. If an atom has more or fewer electrons than its atomic number, then it becomes respectively negatively or positively charged as a whole; a charged atom is called an ion. Electrons have been known since the late 19th century, mostly thanks to J.J. Thomson; see history of subatomic physics for details.

Protons have a positive charge and a mass of 1.6726×10−27 kg. The number of protons in an atom is called its atomic number. Ernest Rutherford (1919) observed that nitrogen under alpha-particle bombardment ejects what appeared to be hydrogen nuclei. By 1920 he had accepted that the hydrogen nucleus is a distinct particle within the atom and named it proton.

Neutrons have no electrical charge and have a mass of 1.6749×10−27 kg.[33][34] Neutrons are the heaviest of the three constituent particles, but their mass can be reduced by the nuclear binding energy. Neutrons and protons (collectively known as nucleons) have comparable dimensions—on the order of 2.5×10−15 m—although the 'surface' of these particles is not sharply defined.[35] The neutron was discovered in 1932 by the English physicist James Chadwick.

In the Standard Model of physics, electrons are truly elementary particles with no internal structure, whereas protons and neutrons are composite particles composed of elementary particles called quarks. There are two types of quarks in atoms, each having a fractional electric charge. Protons are composed of two up quarks (each with charge +⁠
2
/
3
⁠) and one down quark (with a charge of −⁠
1
/
3
⁠). Neutrons consist of one up quark and two down quarks. This distinction accounts for the difference in mass and charge between the two particles.[36][37]

The quarks are held together by the strong interaction (or strong force), which is mediated by gluons. The protons and neutrons, in turn, are held to each other in the nucleus by the nuclear force, which is a residuum of the strong force that has somewhat different range-properties (see the article on the nuclear force for more). The gluon is a member of the family of gauge bosons, which are elementary particles that mediate physical forces.[36][37]

Nucleus
Main article: Atomic nucleus

The binding energy needed for a nucleon to escape the nucleus, for various isotopes
All the bound protons and neutrons in an atom make up a tiny atomic nucleus, and are collectively called nucleons. The radius of a nucleus is approximately equal to 
1.07
𝐴
3
{\displaystyle 1.07{\sqrt[{3}]{A}}} femtometres, where 
𝐴
{\displaystyle A} is the total number of nucleons.[38] This is much smaller than the radius of the atom, which is on the order of 105 fm. The nucleons are bound together by a short-ranged attractive potential called the residual strong force. At distances smaller than 2.5 fm this force is much more powerful than the electrostatic force that causes positively charged protons to repel each other.[39]

Atoms of the same element have the same number of protons, called the atomic number. Within a single element, the number of neutrons may vary, determining the isotope of that element. The total number of protons and neutrons determine the nuclide. The number of neutrons relative to the protons determines the stability of the nucleus, with certain isotopes undergoing radioactive decay.[40]

The proton, the electron, and the neutron are classified as fermions. Fermions obey the Pauli exclusion principle which prohibits identical fermions, such as multiple protons, from occupying the same quantum state at the same time. Thus, every proton in the nucleus must occupy a quantum state different from all other protons, and the same applies to all neutrons of the nucleus and to all electrons of the electron cloud.[41]

A nucleus that has a different number of protons than neutrons can potentially drop to a lower energy state through a radioactive decay that causes the number of protons and neutrons to more closely match. As a result, atoms with matching numbers of protons and neutrons are more stable against decay, but with increasing atomic number, the mutual repulsion of the protons requires an increasing proportion of neutrons to maintain the stability of the nucleus.[41]


Illustration of a nuclear fusion process that forms a deuterium nucleus, consisting of a proton and a neutron, from two protons. A positron (e+)—an antimatter electron—is emitted along with an electron neutrino.
The number of protons and neutrons in the atomic nucleus can be modified, although this can require very high energies because of the strong force. Nuclear fusion occurs when multiple atomic particles join to form a heavier nucleus, such as through the energetic collision of two nuclei. For example, at the core of the Sun protons require energies of 3 to 10 keV to overcome their mutual repulsion—the coulomb barrier—and fuse together into a single nucleus.[42] Nuclear fission is the opposite process, causing a nucleus to split into two smaller nuclei—usually through radioactive decay. The nucleus can also be modified through bombardment by high energy subatomic particles or photons. If this modifies the number of protons in a nucleus, the atom changes to a different chemical element.[43][44]

If the mass of the nucleus following a fusion reaction is less than the sum of the masses of the separate particles, then the difference between these two values can be emitted as a type of usable energy (such as a gamma ray, or the kinetic energy of a beta particle), as described by Albert Einstein's mass–energy equivalence formula, e=mc2, where m is the mass loss and c is the speed of light. This deficit is part of the binding energy of the new nucleus, and it is the non-recoverable loss of the energy that causes the fused particles to remain together in a state that requires this energy to separate.[45]

The fusion of two nuclei that create larger nuclei with lower atomic numbers than iron and nickel—a total nucleon number of about 60—is usually an exothermic process that releases more energy than is required to bring them together.[46] It is this energy-releasing process that makes nuclear fusion in stars a self-sustaining reaction. For heavier nuclei, the binding energy per nucleon begins to decrease. That means that a fusion process producing a nucleus that has an atomic number higher than about 26, and a mass number higher than about 60, is an endothermic process. Thus, more massive nuclei cannot undergo an energy-producing fusion reaction that can sustain the hydrostatic equilibrium of a star.[41]

Electron cloud
Main articles: Electron configuration, Electron shell, and Atomic orbital
See also: Electronegativity

A potential well, showing, according to classical mechanics, the minimum energy V(x) needed to reach each position x. Classically, a particle with energy E is constrained to a range of positions between x1 and x2.
The electrons in an atom are attracted to the protons in the nucleus by the electromagnetic force. This force binds the electrons inside an electrostatic potential well surrounding the smaller nucleus, which means that an external source of energy is needed for the electron to escape. The closer an electron is to the nucleus, the greater the attractive force. Hence electrons bound near the center of the potential well require more energy to escape than those at greater separations.

Electrons, like other particles, have properties of both a particle and a wave. The electron cloud is a region inside the potential well where each electron forms a type of three-dimensional standing wave—a wave form that does not move relative to the nucleus. This behavior is defined by an atomic orbital, a mathematical function that characterises the probability that an electron appears to be at a particular location when its position is measured.[47] Only a discrete (or quantized) set of these orbitals exist around the nucleus, as other possible wave patterns rapidly decay into a more stable form.[48] Orbitals can have one or more ring or node structures, and differ from each other in size, shape and orientation.[49]


3D views of some hydrogen-like atomic orbitals showing probability density and phase (g orbitals and higher are not shown)
Each atomic orbital corresponds to a particular energy level of the electron. The electron can change its state to a higher energy level by absorbing a photon with sufficient energy to boost it into the new quantum state. Likewise, through spontaneous emission, an electron in a higher energy state can drop to a lower energy state while radiating the excess energy as a photon. These characteristic energy values, defined by the differences in the energies of the quantum states, are responsible for atomic spectral lines.[48]

The amount of energy needed to remove or add an electron—the electron binding energy—is far less than the binding energy of nucleons. For example, it requires only 13.6 eV to strip a ground-state electron from a hydrogen atom,[50] compared to 2.23 million eV for splitting a deuterium nucleus.[51] Atoms are electrically neutral if they have an equal number of protons and electrons. Atoms that have either a deficit or a surplus of electrons are called ions. Electrons that are farthest from the nucleus may be transferred to other nearby atoms or shared between atoms. By this mechanism, atoms are able to bond into molecules and other types of chemical compounds like ionic and covalent network crystals.[52]

Properties
Nuclear properties
Main articles: Isotope, Stable isotope, List of nuclides, and List of elements by stability of isotopes
By definition, any two atoms with an identical number of protons in their nuclei belong to the same chemical element. Atoms with equal numbers of protons but a different number of neutrons are different isotopes of the same element. For example, all hydrogen atoms admit exactly one proton, but isotopes exist with no neutrons (hydrogen-1, by far the most common form,[53] also called protium), one neutron (deuterium), two neutrons (tritium) and more than two neutrons. The known elements form a set of atomic numbers, from the single-proton element hydrogen up to the 118-proton element oganesson.[54] All known isotopes of elements with atomic numbers greater than 82 are radioactive, although the radioactivity of element 83 (bismuth) is so slight as to be practically negligible.[55][56]

About 339 nuclides occur naturally on Earth,[57] of which 251 (about 74%) have not been observed to decay, and are referred to as "stable isotopes". Only 90 nuclides are stable theoretically, while another 161 (bringing the total to 251) have not been observed to decay, even though in theory it is energetically possible. These are also formally classified as "stable". An additional 35 radioactive nuclides have half-lives longer than 100 million years, and are long-lived enough to have been present since the birth of the Solar System. This collection of 286 nuclides are known as primordial nuclides. Finally, an additional 53 short-lived nuclides are known to occur naturally, as daughter products of primordial nuclide decay (such as radium from uranium), or as products of natural energetic processes on Earth, such as cosmic ray bombardment (for example, carbon-14).[58][note 1]

For 80 of the chemical elements, at least one stable isotope exists. As a rule, there is only a handful of stable isotopes for each of these elements, the average being 3.1 stable isotopes per element. Twenty-six "monoisotopic elements" have only a single stable isotope, while the largest number of stable isotopes observed for any element is ten, for the element tin. Elements 43, 61, and all elements numbered 83 or higher have no stable isotopes.[59]: 1–12 

Stability of isotopes is affected by the ratio of protons to neutrons, and also by the presence of certain "magic numbers" of neutrons or protons that represent closed and filled quantum shells. These quantum shells correspond to a set of energy levels within the shell model of the nucleus; filled shells, such as the filled shell of 50 protons for tin, confers unusual stability on the nuclide. Of the 251 known stable nuclides, only four have both an odd number of protons and odd number of neutrons: hydrogen-2 (deuterium), lithium-6, boron-10, and nitrogen-14. (Tantalum-180m is odd-odd and observationally stable, but is predicted to decay with a very long half-life.) Also, only four naturally occurring, radioactive odd-odd nuclides have a half-life over a billion years: potassium-40, vanadium-50, lanthanum-138, and lutetium-176. Most odd-odd nuclei are highly unstable with respect to beta decay, because the decay products are even-even, and are therefore more strongly bound, due to nuclear pairing effects.[60]

Mass
Main articles: Atomic mass and mass number
The large majority of an atom's mass comes from the protons and neutrons that make it up. The total number of these particles (called "nucleons") in a given atom is called the mass number. It is a positive integer and dimensionless (instead of having dimension of mass), because it expresses a count. An example of use of a mass number is "carbon-12," which has 12 nucleons (six protons and six neutrons).

The actual mass of an atom at rest is often expressed in daltons (Da), also called the unified atomic mass unit (u). This unit is defined as a twelfth of the mass of a free neutral atom of carbon-12, which is approximately 1.66×10−27 kg.[61] Hydrogen-1 (the lightest isotope of hydrogen which is also the nuclide with the lowest mass) has an atomic weight of 1.007825 Da.[62] The value of this number is called the atomic mass. A given atom has an atomic mass approximately equal (within 1%) to its mass number times the atomic mass unit (for example the mass of a nitrogen-14 is roughly 14 Da), but this number will not be exactly an integer except (by definition) in the case of carbon-12.[63] The heaviest stable atom is lead-208,[55] with a mass of 207.9766521 Da.[64]

As even the most massive atoms are far too light to work with directly, chemists instead use the unit of moles. One mole of atoms of any element always has the same number of atoms (about 6.022×1023). This number was chosen so that if an element has an atomic mass of 1 u, a mole of atoms of that element has a mass close to one gram. Because of the definition of the unified atomic mass unit, each carbon-12 atom has an atomic mass of exactly 12 Da, and so a mole of carbon-12 atoms weighs exactly 0.012 kg.[61]

Shape and size
Main article: Atomic radius
Atoms lack a well-defined outer boundary, so their dimensions are usually described in terms of an atomic radius. This is a measure of the distance out to which the electron cloud extends from the nucleus.[65] This assumes the atom to exhibit a spherical shape, which is only obeyed for atoms in vacuum or free space. Atomic radii may be derived from the distances between two nuclei when the two atoms are joined in a chemical bond. The radius varies with the location of an atom on the atomic chart, the type of chemical bond, the number of neighboring atoms (coordination number) and a quantum mechanical property known as spin.[66] On the periodic table of the elements, atom size tends to increase when moving down columns, but decrease when moving across rows (left to right).[67] Consequently, the smallest atom is helium with a radius of 32 pm, while one of the largest is caesium at 225 pm.[68]

When subjected to external forces, like electrical fields, the shape of an atom may deviate from spherical symmetry. The deformation depends on the field magnitude and the orbital type of outer shell electrons, as shown by group-theoretical considerations. Aspherical deviations might be elicited for instance in crystals, where large crystal-electrical fields may occur at low-symmetry lattice sites.[69][70] Significant ellipsoidal deformations have been shown to occur for sulfur ions[71] and chalcogen ions[72] in pyrite-type compounds.

Atomic dimensions are thousands of times smaller than the wavelengths of light (400–700 nm) so they cannot be viewed using an optical microscope, although individual atoms can be observed using a scanning tunneling microscope. To visualize the minuteness of the atom, consider that a typical human hair is about 1 million carbon atoms in width.[73] A single drop of water contains about 2 sextillion (2×1021) atoms of oxygen, and twice the number of hydrogen atoms.[74] A single carat diamond with a mass of 2×10−4 kg contains about 10 sextillion (1022) atoms of carbon.[note 2] If an apple were magnified to the size of the Earth, then the atoms in the apple would be approximately the size of the original apple.[75]

Radioactive decay
Main article: Radioactive decay

This diagram shows the half-life (T1⁄2) of various isotopes with Z protons and N neutrons.
Every element has one or more isotopes that have unstable nuclei that are subject to radioactive decay, causing the nucleus to emit particles or electromagnetic radiation. Radioactivity can occur when the radius of a nucleus is large compared with the radius of the strong force, which only acts over distances on the order of 1 fm.[76]

The most common forms of radioactive decay are:[77][78]

Alpha decay: this process is caused when the nucleus emits an alpha particle, which is a helium nucleus consisting of two protons and two neutrons. The result of the emission is a new element with a lower atomic number.
Beta decay (and electron capture): these processes are regulated by the weak force, and result from a transformation of a neutron into a proton, or a proton into a neutron. The neutron to proton transition is accompanied by the emission of an electron and an antineutrino, while proton to neutron transition (except in electron capture) causes the emission of a positron and a neutrino. The electron or positron emissions are called beta particles. Beta decay either increases or decreases the atomic number of the nucleus by one. Electron capture is more common than positron emission, because it requires less energy. In this type of decay, an electron is absorbed by the nucleus, rather than a positron emitted from the nucleus. A neutrino is still emitted in this process, and a proton changes to a neutron.
Gamma decay: this process results from a change in the energy level of the nucleus to a lower state, resulting in the emission of electromagnetic radiation. The excited state of a nucleus which results in gamma emission usually occurs following the emission of an alpha or a beta particle. Thus, gamma decay usually follows alpha or beta decay.
Other more rare types of radioactive decay include ejection of neutrons or protons or clusters of nucleons from a nucleus, or more than one beta particle. An analog of gamma emission which allows excited nuclei to lose energy in a different way, is internal conversion—a process that produces high-speed electrons that are not beta rays, followed by production of high-energy photons that are not gamma rays. A few large nuclei explode into two or more charged fragments of varying masses plus several neutrons, in a decay called spontaneous nuclear fission.

Each radioactive isotope has a characteristic decay time period—the half-life—that is determined by the amount of time needed for half of a sample to decay. This is an exponential decay process that steadily decreases the proportion of the remaining isotope by 50% every half-life. Hence after two half-lives have passed only 25% of the isotope is present, and so forth.[76]

Magnetic moment
Main articles: Electron magnetic moment and Nuclear magnetic moment
Elementary particles possess an intrinsic quantum mechanical property known as spin. This is analogous to the angular momentum of an object that is spinning around its center of mass, although strictly speaking these particles are believed to be point-like and cannot be said to be rotating. Spin is measured in units of the reduced Planck constant (ħ), with electrons, protons and neutrons all having spin 1⁄2 ħ, or "spin-1⁄2". In an atom, electrons in motion around the nucleus possess orbital angular momentum in addition to their spin, while the nucleus itself possesses angular momentum due to its nuclear spin.[79]

The magnetic field produced by an atom—its magnetic moment—is determined by these various forms of angular momentum, just as a rotating charged object classically produces a magnetic field, but the most dominant contribution comes from electron spin. Due to the nature of electrons to obey the Pauli exclusion principle, in which no two electrons may be found in the same quantum state, bound electrons pair up with each other, with one member of each pair in a spin up state and the other in the opposite, spin down state. Thus these spins cancel each other out, reducing the total magnetic dipole moment to zero in some atoms with even number of electrons.[80]

In ferromagnetic elements such as iron, cobalt and nickel, an odd number of electrons leads to an unpaired electron and a net overall magnetic moment. The orbitals of neighboring atoms overlap and a lower energy state is achieved when the spins of unpaired electrons are aligned with each other, a spontaneous process known as an exchange interaction. When the magnetic moments of ferromagnetic atoms are lined up, the material can produce a measurable macroscopic field. Paramagnetic materials have atoms with magnetic moments that line up in random directions when no magnetic field is present, but the magnetic moments of the individual atoms line up in the presence of a field.[80][81]

The nucleus of an atom will have no spin when it has even numbers of both neutrons and protons, but for other cases of odd numbers, the nucleus may have a spin. Normally nuclei with spin are aligned in random directions because of thermal equilibrium, but for certain elements (such as xenon-129) it is possible to polarize a significant proportion of the nuclear spin states so that they are aligned in the same direction—a condition called hyperpolarization. This has important applications in magnetic resonance imaging.[82][83]

Energy levels

These electron's energy levels (not to scale) are sufficient for ground states of atoms up to cadmium (5s2 4d10) inclusively. Do not forget that even the top of the diagram is lower than an unbound electron state.
The potential energy of an electron in an atom is negative relative to when the distance from the nucleus goes to infinity; its dependence on the electron's position reaches the minimum inside the nucleus, roughly in inverse proportion to the distance. In the quantum-mechanical model, a bound electron can occupy only a set of states centered on the nucleus, and each state corresponds to a specific energy level; see time-independent Schrödinger equation for a theoretical explanation. An energy level can be measured by the amount of energy needed to unbind the electron from the atom, and is usually given in units of electronvolts (eV). The lowest energy state of a bound electron is called the ground state, i.e. stationary state, while an electron transition to a higher level results in an excited state.[84] The electron's energy increases along with n because the (average) distance to the nucleus increases. Dependence of the energy on ℓ is caused not by the electrostatic potential of the nucleus, but by interaction between electrons.

For an electron to transition between two different states, e.g. ground state to first excited state, it must absorb or emit a photon at an energy matching the difference in the potential energy of those levels, according to the Niels Bohr model, what can be precisely calculated by the Schrödinger equation. Electrons jump between orbitals in a particle-like fashion. For example, if a single photon strikes the electrons, only a single electron changes states in response to the photon; see Electron properties.

The energy of an emitted photon is proportional to its frequency, so these specific energy levels appear as distinct bands in the electromagnetic spectrum.[85] Each element has a characteristic spectrum that can depend on the nuclear charge, subshells filled by electrons, the electromagnetic interactions between the electrons and other factors.[86]


An example of absorption lines in a spectrum
When a continuous spectrum of energy is passed through a gas or plasma, some of the photons are absorbed by atoms, causing electrons to change their energy level. Those excited electrons that remain bound to their atom spontaneously emit this energy as a photon, traveling in a random direction, and so drop back to lower energy levels. Thus the atoms behave like a filter that forms a series of dark absorption bands in the energy output. (An observer viewing the atoms from a view that does not include the continuous spectrum in the background, instead sees a series of emission lines from the photons emitted by the atoms.) Spectroscopic measurements of the strength and width of atomic spectral lines allow the composition and physical properties of a substance to be determined.[87]

Close examination of the spectral lines reveals that some display a fine structure splitting. This occurs because of spin–orbit coupling, which is an interaction between the spin and motion of the outermost electron.[88] When an atom is in an external magnetic field, spectral lines become split into three or more components; a phenomenon called the Zeeman effect. This is caused by the interaction of the magnetic field with the magnetic moment of the atom and its electrons. Some atoms can have multiple electron configurations with the same energy level, which thus appear as a single spectral line. The interaction of the magnetic field with the atom shifts these electron configurations to slightly different energy levels, resulting in multiple spectral lines.[89] The presence of an external electric field can cause a comparable splitting and shifting of spectral lines by modifying the electron energy levels, a phenomenon called the Stark effect.[90]

If a bound electron is in an excited state, an interacting photon with the proper energy can cause stimulated emission of a photon with a matching energy level. For this to occur, the electron must drop to a lower energy state that has an energy difference matching the energy of the interacting photon. The emitted photon and the interacting photon then move off in parallel and with matching phases. That is, the wave patterns of the two photons are synchronized. This physical property is used to make lasers, which can emit a coherent beam of light energy in a narrow frequency band.[91]

Valence and bonding behavior
Main articles: Valence (chemistry) and Chemical bond
Valency is the combining power of an element. It is determined by the number of bonds it can form to other atoms or groups.[92] The outermost electron shell of an atom in its uncombined state is known as the valence shell, and the electrons in that shell are called valence electrons. The number of valence electrons determines the bonding behavior with other atoms. Atoms tend to chemically react with each other in a manner that fills (or empties) their outer valence shells.[93] For example, a transfer of a single electron between atoms is a useful approximation for bonds that form between atoms with one-electron more than a filled shell, and others that are one-electron short of a full shell, such as occurs in the compound sodium chloride and other chemical ionic salts. Many elements display multiple valences, or tendencies to share differing numbers of electrons in different compounds. Thus, chemical bonding between these elements takes many forms of electron-sharing that are more than simple electron transfers. Examples include the element carbon and the organic compounds.[94]

The chemical elements are often displayed in a periodic table that is laid out to display recurring chemical properties, and elements with the same number of valence electrons form a group that is aligned in the same column of the table. (The horizontal rows correspond to the filling of a quantum shell of electrons.) The elements at the far right of the table have their outer shell completely filled with electrons, which results in chemically inert elements known as the noble gases.[95][96]

States
Main articles: State of matter and Phase (matter)

Graphic illustrating the formation of a Bose–Einstein condensate
Quantities of atoms are found in different states of matter that depend on the physical conditions, such as temperature and pressure. By varying the conditions, materials can transition between solids, liquids, gases and plasmas.[97] Within a state, a material can also exist in different allotropes. An example of this is solid carbon, which can exist as graphite or diamond.[98] Gaseous allotropes exist as well, such as dioxygen and ozone.

At temperatures close to absolute zero, atoms can form a Bose–Einstein condensate, at which point quantum mechanical effects, which are normally only observed at the atomic scale, become apparent on a macroscopic scale.[99][100] This super-cooled collection of atoms then behaves as a single super atom, which may allow fundamental checks of quantum mechanical behavior.[101]

Identification

Scanning tunneling microscope image showing the individual atoms making up this gold (100) surface. The surface atoms deviate from the bulk crystal structure and arrange in columns several atoms wide with pits between them (See surface reconstruction).
While atoms are too small to be seen, devices such as the scanning tunneling microscope (STM) enable their visualization at the surfaces of solids. The microscope uses the quantum tunneling phenomenon, which allows particles to pass through a barrier that would be insurmountable in the classical perspective. Electrons tunnel through the vacuum between two biased electrodes, providing a tunneling current that is exponentially dependent on their separation. One electrode is a sharp tip ideally ending with a single atom. At each point of the scan of the surface the tip's height is adjusted so as to keep the tunneling current at a set value. How much the tip moves to and away from the surface is interpreted as the height profile. For low bias, the microscope images the averaged electron orbitals across closely packed energy levels—the local density of the electronic states near the Fermi level.[102][103] Because of the distances involved, both electrodes need to be extremely stable; only then periodicities can be observed that correspond to individual atoms. The method alone is not chemically specific, and cannot identify the atomic species present at the surface.

Atoms can be easily identified by their mass. If an atom is ionized by removing one of its electrons, its trajectory when it passes through a magnetic field will bend. The radius by which the trajectory of a moving ion is turned by the magnetic field is determined by the mass of the atom. The mass spectrometer uses this principle to measure the mass-to-charge ratio of ions. If a sample contains multiple isotopes, the mass spectrometer can determine the proportion of each isotope in the sample by measuring the intensity of the different beams of ions. Techniques to vaporize atoms include inductively coupled plasma atomic emission spectroscopy and inductively coupled plasma mass spectrometry, both of which use a plasma to vaporize samples for analysis.[104]

The atom-probe tomograph has sub-nanometer resolution in 3-D and can chemically identify individual atoms using time-of-flight mass spectrometry.[105]

Electron emission techniques such as X-ray photoelectron spectroscopy (XPS) and Auger electron spectroscopy (AES), which measure the binding energies of the core electrons, are used to identify the atomic species present in a sample in a non-destructive way. With proper focusing both can be made area-specific. Another such method is electron energy loss spectroscopy (EELS), which measures the energy loss of an electron beam within a transmission electron microscope when it interacts with a portion of a sample.

Spectra of excited states can be used to analyze the atomic composition of distant stars. Specific light wavelengths contained in the observed light from stars can be separated out and related to the quantized transitions in free gas atoms. These colors can be replicated using a gas-discharge lamp containing the same element.[106] Helium was discovered in this way in the spectrum of the Sun 23 years before it was found on Earth.[107]

Origin and current state
Baryonic matter forms about 4% of the total energy density of the observable universe, with an average density of about 0.25 particles/m3 (mostly protons and electrons).[108] Within a galaxy such as the Milky Way, particles have a much higher concentration, with the density of matter in the interstellar medium (ISM) ranging from 105 to 109 atoms/m3.[109] The Sun is believed to be inside the Local Bubble, so the density in the solar neighborhood is only about 103 atoms/m3.[110] Stars form from dense clouds in the ISM, and the evolutionary processes of stars result in the steady enrichment of the ISM with elements more massive than hydrogen and helium.

Up to 95% of the Milky Way's baryonic matter are concentrated inside stars, where conditions are unfavorable for atomic matter. The total baryonic mass is about 10% of the mass of the galaxy;[111] the remainder of the mass is an unknown dark matter.[112] High temperature inside stars makes most "atoms" fully ionized, that is, separates all electrons from the nuclei. In stellar remnants—with exception of their surface layers—an immense pressure make electron shells impossible.

Formation
Main article: Nucleosynthesis

Periodic table showing the origin of each element. Elements from carbon up to sulfur may be made in small stars by the alpha process. Elements beyond iron are made in large stars with slow neutron capture (s-process). Elements heavier than iron may be made in neutron star mergers or supernovae after the r-process.
Electrons are thought to exist in the Universe since early stages of the Big Bang. Atomic nuclei forms in nucleosynthesis reactions. In about three minutes Big Bang nucleosynthesis produced most of the helium, lithium, and deuterium in the Universe, and perhaps some of the beryllium and boron.[113][114][115]

Ubiquitousness and stability of atoms relies on their binding energy, which means that an atom has a lower energy than an unbound system of the nucleus and electrons. Where the temperature is much higher than ionization potential, the matter exists in the form of plasma—a gas of positively charged ions (possibly, bare nuclei) and electrons. When the temperature drops below the ionization potential, atoms become statistically favorable. Atoms (complete with bound electrons) became to dominate over charged particles 380,000 years after the Big Bang—an epoch called recombination, when the expanding Universe cooled enough to allow electrons to become attached to nuclei.[116]

Since the Big Bang, which produced no carbon or heavier elements, atomic nuclei have been combined in stars through the process of nuclear fusion to produce more of the element helium, and (via the triple-alpha process) the sequence of elements from carbon up to iron;[117] see stellar nucleosynthesis for details.

Isotopes such as lithium-6, as well as some beryllium and boron are generated in space through cosmic ray spallation.[118] This occurs when a high-energy proton strikes an atomic nucleus, causing large numbers of nucleons to be ejected.

Elements heavier than iron were produced in supernovae and colliding neutron stars through the r-process, and in AGB stars through the s-process, both of which involve the capture of neutrons by atomic nuclei.[119] Elements such as lead formed largely through the radioactive decay of heavier elements.[120]

Earth
Most of the atoms that make up the Earth and its inhabitants were present in their current form in the nebula that collapsed out of a molecular cloud to form the Solar System. The rest are the result of radioactive decay, and their relative proportion can be used to determine the age of the Earth through radiometric dating.[121][122] Most of the helium in the crust of the Earth (about 99% of the helium from gas wells, as shown by its lower abundance of helium-3) is a product of alpha decay.[123]

There are a few trace atoms on Earth that were not present at the beginning (i.e., not "primordial"), nor are results of radioactive decay. Carbon-14 is continuously generated by cosmic rays in the atmosphere.[124] Some atoms on Earth have been artificially generated either deliberately or as by-products of nuclear reactors or explosions.[125][126] Of the transuranic elements—those with atomic numbers greater than 92—only plutonium and neptunium occur naturally on Earth.[127][128] Transuranic elements have radioactive lifetimes shorter than the current age of the Earth[129] and thus identifiable quantities of these elements have long since decayed, with the exception of traces of plutonium-244 possibly deposited by cosmic dust.[121] Natural deposits of plutonium and neptunium are produced by neutron capture in uranium ore.[130]

The Earth contains approximately 1.33×1050 atoms.[131] Although small numbers of independent atoms of noble gases exist, such as argon, neon, and helium, 99% of the atmosphere is bound in the form of molecules, including carbon dioxide and diatomic oxygen and nitrogen. At the surface of the Earth, an overwhelming majority of atoms combine to form various compounds, including water, salt, silicates and oxides. Atoms can also combine to create materials that do not consist of discrete molecules, including crystals and liquid or solid metals.[132][133] This atomic matter forms networked arrangements that lack the particular type of small-scale interrupted order associated with molecular matter.[134]

Rare and theoretical forms
Superheavy elements
Main article: Superheavy element
All nuclides with atomic numbers higher than 82 (lead) are known to be radioactive. No nuclide with an atomic number exceeding 92 (uranium) exists on Earth as a primordial nuclide, and heavier elements generally have shorter half-lives. Nevertheless, an "island of stability" encompassing relatively long-lived isotopes of superheavy elements[135] with atomic numbers 110 to 114 might exist.[136] Predictions for the half-life of the most stable nuclide on the island range from a few minutes to millions of years.[137] In any case, superheavy elements (with Z > 104) would not exist due to increasing Coulomb repulsion (which results in spontaneous fission with increasingly short half-lives) in the absence of any stabilizing effects.[138]

Exotic matter
Main article: Exotic matter
Each particle of matter has a corresponding antimatter particle with the opposite electrical charge. Thus, the positron is a positively charged antielectron and the antiproton is a negatively charged equivalent of a proton. When a matter and corresponding antimatter particle meet, they annihilate each other. Because of this, along with an imbalance between the number of matter and antimatter particles, the latter are rare in the universe. The first causes of this imbalance are not yet fully understood, although theories of baryogenesis may offer an explanation. As a result, no antimatter atoms have been discovered in nature.[139][140] In 1996, the antimatter counterpart of the hydrogen atom (antihydrogen) was synthesized at the CERN laboratory in Geneva.[141][142]

Other exotic atoms have been created by replacing one of the protons, neutrons or electrons with other particles that have the same charge. For example, an electron can be replaced by a more massive muon, forming a muonic atom. These types of atoms can be used to test fundamental predictions of physics.[143][144][145]


"""

In [16]:
chunk_from_an_article = "More than 99.94% of an atom's mass is in the nucleus. Protons have a positive electric charge and neutrons have no charge, so the nucleus is positively charged. The electrons are negatively charged, and this opposing charge is what binds them to the nucleus."

In [17]:
message = f"""
<document>
{article}
</document>
Here is a chunk we want to situate within the whole document
<chunk>
{chunk_from_an_article}
</chunk>
Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrival of the chunk. Answer only with the succinct context and nothing else.
"""

In [ ]:
L_calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY, max_tokens=2000)

L_calls.chat(message)

This chunk appears in the early part of the document introducing the basic structure of atoms. It describes the mass distribution and electrical charges of the main components of an atom - the nucleus (containing protons and neutrons) and the surrounding electrons.

Now try to make LLM pipeline using this technique (as we did before)

#### Example of pipeline for this technique: 
Whenever you load a document to you application you split it to the chunks and for each chunk generate such small description to situate it and the save to DB in such way: (chunk_description + '\n\n' + chunk).
